_________
# Phase 2 - Milestone 1
    Pratama Waskito Prihandana
    FTDS - Batch 003

    Model Deployment :

_____
# I. Library

In [1]:
!pip install feature_engine
!pip install phik
!pip install scikit-learn
!pip install numpy

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from scipy import stats

from tensorflow import keras
from keras.wrappers.scikit_learn import KerasClassifier

# correlation finder
import phik
from phik import resources, report
from phik.report import plot_correlation_matrix

# hyperparameter tuning
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

# preprocessing data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

# model evaluation
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,precision_recall_curve,auc,f1_score

# sequential model to initialise ann and dense module to build the layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop
from keras.optimizers import Adadelta
from keras.optimizers import Adagrad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 KB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


_____
# II. Load Dataset

In [5]:
from google.colab import files
uploaded = files.upload()

Saving xdata_inference_p2ms1.csv to xdata_inference_p2ms1.csv


In [6]:
df_inf = pd.read_csv('xdata_inference_p2ms1.csv')
df_inf_backup = df_inf

In [8]:
df_inf.head()

,Unnamed: 0,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,Village,Platinum Membership,No,Credit/Debit Card Offers,Credit/Debit Card Offers,Wi-Fi,124.40,63557.17,36.0,1932.37,No,No,No,Solved in Follow-up,User Friendly Website,0
1,1,Village,Silver Membership,Yes,Credit/Debit Card Offers,Without Offers,Mobile_Data,635.20,30402.83,50.0,499.19,No,Yes,Yes,Solved,Poor Website,1
2,2,Village,Premium Membership,Yes,Gift Vouchers/Coupons,Without Offers,Fiber_Optic,1544.34,69727.67,53.0,823.79,No,Yes,Yes,No Information Available,Quality Customer Care,1
3,3,Village,Silver Membership,No,Without Offers,Gift Vouchers/Coupons,Wi-Fi,216.65,33500.64,18.0,1941.21,No,No,Yes,No Information Available,Quality Customer Care,1
4,4,City,Gold Membership,Yes,Credit/Debit Card Offers,Without Offers,Mobile_Data,2895.90,80535.39,65.0,2041.86,Yes,No,Yes,Solved in Follow-up,No reason specified,1


In [9]:
df_inf.tail()

,Unnamed: 0,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
295,295,Village,Premium Membership,Yes,Without Offers,Without Offers,Fiber_Optic,246.15,26476.99,7.0,880.70,Yes,No,No,Not Applicable,Products always in Stock,1
296,296,Town,Silver Membership,Yes,Gift Vouchers/Coupons,Credit/Debit Card Offers,Wi-Fi,1212.44,79910.59,15.0,200.23,No,Yes,No,Not Applicable,Poor Website,0
297,297,City,Platinum Membership,No,Without Offers,Gift Vouchers/Coupons,Fiber_Optic,1992.17,75245.69,29.0,1242.96,No,Yes,No,Unsolved,Reasonable Price,1
298,298,Town,Silver Membership,No,Gift Vouchers/Coupons,Without Offers,Mobile_Data,430.34,3233.66,30.0,1986.95,No,Yes,No,No Information Available,Too many ads,1
299,299,Town,Basic Membership,Yes,Without Offers,Credit/Debit Card Offers,Wi-Fi,596.96,56071.62,42.0,341.82,No,No,No,No Information Available,Poor Product Quality,0


In [10]:
# drop unnecessary column

df_inf = df_inf.drop('Unnamed: 0', axis=1)

______
# III. Model Implementation

In [11]:
df_inf.shape

(300, 16)

In [12]:
numlist = ['avg_time_spent',
           'avg_transaction_value',
           'avg_frequency_login_days',
           'points_in_wallet']

catlist = [ 'region_category',
            'membership_category',
            'joined_through_referral',
            'preferred_offer_types',
            'medium_of_operation',
            'internet_option',
            'used_special_discount',
            'offer_application_preference',
            'past_complaint',
            'complaint_status',
            'feedback']

In [30]:
# scaling numerical data
scaler = MinMaxScaler()
df_inf_num_scaled = pd.DataFrame(scaler.fit_transform(df_inf[numlist]))

# encoding categorical data
encode = OrdinalEncoder()
df_inf_cat_encoded = pd.DataFrame(encode.fit_transform(df_inf[catlist]))

# concatenate numerical and categorical
df_inf2 = pd.DataFrame(np.concatenate([df_inf_num_scaled, df_inf_cat_encoded], axis=1), columns=df_inf[numlist].columns.tolist()+df_inf[catlist].columns.tolist()).round(2)



In [31]:
df_inf2.shape

(300, 15)

In [32]:
df_inf2.columns.tolist()

['avg_time_spent',
 'avg_transaction_value',
 'avg_frequency_login_days',
 'points_in_wallet',
 'region_category',
 'membership_category',
 'joined_through_referral',
 'preferred_offer_types',
 'medium_of_operation',
 'internet_option',
 'used_special_discount',
 'offer_application_preference',
 'past_complaint',
 'complaint_status',
 'feedback']

In [33]:
df_inf2.head()

,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,0.03,0.63,0.49,0.94,2.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,8.0
1,0.19,0.30,0.68,0.24,2.0,5.0,1.0,0.0,2.0,1.0,0.0,1.0,1.0,2.0,3.0
2,0.48,0.69,0.73,0.40,2.0,4.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,5.0
3,0.06,0.33,0.25,0.94,2.0,5.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,0.0,5.0
4,0.90,0.80,0.89,0.99,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,3.0,0.0


In [20]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

if len(uploaded) >= 2:
 
  model1 = tf.keras.models.load_model(list(uploaded.keys())[0])

  model2 = tf.keras.models.load_model(list(uploaded.keys())[1])
  
else:
  print("Error: At least two .h5 files are required to load two different models.")


Saving model1_ann_sequential.h5 to model1_ann_sequential (4).h5
Saving model4_ann_functional.h5 to model4_ann_functional.h5
User uploaded file "model1_ann_sequential.h5" with length 41512 bytes
User uploaded file "model4_ann_functional.h5" with length 64144 bytes


### Model 1

In [41]:
model1_pred = pd.DataFrame(model1.predict(df_inf2))
model1_pred = (model1_pred > 0.5)

10/10 [==============================] - 0s 3ms/step


In [45]:
model1_pred.head()

,0
0,False
1,True
2,False
3,False
4,False


### Model 2

In [43]:
model2_pred = pd.DataFrame(model2.predict(df_inf2))
model2_pred = (model2_pred > 0.5)

10/10 [==============================] - 0s 2ms/step


In [46]:
model2_pred.head()

,0
0,False
1,True
2,False
3,False
4,False


### Concatenate prediction result with dataset inference

In [49]:
prediction_result = pd.DataFrame(np.concatenate([df_inf, model1_pred, model2_pred],axis=1), columns=df_inf.columns.tolist()+['model1_pred']+['model2_pred'])
prediction_result.head()

,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,model1_pred,model2_pred
0,Village,Platinum Membership,No,Credit/Debit Card Offers,Credit/Debit Card Offers,Wi-Fi,124.4,63557.17,36.0,1932.37,No,No,No,Solved in Follow-up,User Friendly Website,0,False,False
1,Village,Silver Membership,Yes,Credit/Debit Card Offers,Without Offers,Mobile_Data,635.2,30402.83,50.0,499.19,No,Yes,Yes,Solved,Poor Website,1,True,True
2,Village,Premium Membership,Yes,Gift Vouchers/Coupons,Without Offers,Fiber_Optic,1544.34,69727.67,53.0,823.79,No,Yes,Yes,No Information Available,Quality Customer Care,1,False,False
3,Village,Silver Membership,No,Without Offers,Gift Vouchers/Coupons,Wi-Fi,216.65,33500.64,18.0,1941.21,No,No,Yes,No Information Available,Quality Customer Care,1,False,False
4,City,Gold Membership,Yes,Credit/Debit Card Offers,Without Offers,Mobile_Data,2895.9,80535.39,65.0,2041.86,Yes,No,Yes,Solved in Follow-up,No reason specified,1,False,False
